<a href="https://colab.research.google.com/github/alessiomongoli/Sentiment_Lexicon/blob/main/FileGenerale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/alessiomongoli/Sentiment_Lexicon.git

Cloning into 'Sentiment_Lexicon'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 128 (delta 63), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (128/128), 45.67 KiB | 11.42 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import import_ipynb
import sys
sys.path.append('/content/Sentiment_Lexicon/')

In [ ]:
%cd /content/Sentiment_Lexicon/

/content/Sentiment_Lexicon


In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Grocery_and_Gourmet_Food.json.gz


In [ ]:
!gdown https://drive.google.com/uc?id=1mpL3_URUIDlY0I5MiaCi_tWXqGdA2-6k

In [8]:
!gdown https://drive.google.com/u/0/uc?id=1PQ51btvMqsmhNQEUhxaOKlfaL2K-pSV2

Downloading...
From: https://drive.google.com/u/0/uc?id=1PQ51btvMqsmhNQEUhxaOKlfaL2K-pSV2
To: /content/IMDB-Dataset_preprocessed.csv
100% 64.7M/64.7M [00:00<00:00, 279MB/s]


In [ ]:
import pandas as pd
import numpy as np


from nltk.corpus import stopwords
import nltk 
import string


from Data_induction.PreProcessing import preprocessing
from Data_induction.TrainSVM import LinearCoefficentsSVM
from glove.Glove import load_filter_glove_word
from Data_induction.Seed_Data import SeedDataset
from Neural_model.EarlyStopping import EarlyStopping
from Neural_model.Neural import RegressionModel
from Neural_model.Train_predict_NN import train, predict
from Utils.DataframeCreation import creation_dataframe
from Test.Experiment1 import experiment1


In [ ]:
CATEGORY="reviews_Grocery_and_Gourmet_Food" #"reviews_Apps_for_Android" #reviews_Apps_for_Android.json.gz or reviews_Grocery_and_Gourmet_Food.json.gz or reviews_Movies_and_TV.json.gz
SAVE_DATAFRAME_FILE= False
CHECKPOINT_PATH = "/content/Sentiment_Lexicon/checkpoint/checkpoint.pt"
NEGATION_TYPE='all_words' # 'normal' or 'all_words' or 'no_negation'
NEGATION_MEAN=True
REPOSITORY = '/content/Sentiment_Lexicon/'

In [ ]:
"""" Preprocessing function:
1 Apre il file dove ci sono Recensioni e score.
2 Filtra le recensioni togliendo quelle con 3 stelle
3Usa la funzione find negation per aggiungere dei tag nelle recensioni dove trova vocaboli negativi {not never nor}
4)Crea due liste:recensioni, label (1 se >3 stelle, -1< 3 stelle)
5) Genera il document term Matrix 
6) Ritorna:
X Bag of word
Y array di label 1 o 0
Frequenze lista con le occurences
Vocabulary 
"""

X, y, frequencies, vocabulary = preprocessing(REPOSITORY+CATEGORY+".json.gz", NEGATION_TYPE, CATEGORY)

In [ ]:
"""
Fa il training di SVM con le BOW e le label ed estrae i pesi che vengono considerate come polarità
"""
coefficents = LinearCoefficentsSVM(X, y, CATEGORY, NEGATION_TYPE)

In [ ]:
Data=creation_dataframe(vocabulary,coefficents, frequencies)
if SAVE_DATAFRAME_FILE==True:
  Data.to_csv('/content/Sentiment_Lexicon/Data/'+CATEGORY+'_'+NEGATION_TYPE+'.csv',index=False)

In [ ]:
if SAVE_DATAFRAME_FILE==True:
  Data = pd.read_csv(REPOSITORY+'Data/'+CATEGORY+'_'+NEGATION_TYPE+'.csv')
Data

In [ ]:
Data = Data[Data['Frequence']>=500]
Data.reset_index(drop=True, inplace=True)

In [ ]:
Data

When Negation_MEAN is true we consider also the Neg_word  adjust polarity of the word without Neg:
# New Polarity [word]=(Polarity[word]+ Polarity[NEG_word])/2

In [ ]:
if NEGATION_MEAN:
  negated={}
  prefix=len('neg_')
  for i in range(len(Data)):
    if str(Data['Token'][i]).startswith('neg_')==True:
      negated[Data['Token'][i][prefix:]]=Data['Polarity'][i]
  for i in range(len(Data)):
    if Data['Token'][i]in negated.keys():
      Data['Polarity'][i]=(negated[Data['Token'][i]]+Data['Polarity'][i]/2)

In [ ]:
Data

In [ ]:
%%time
#glove_embeddings_filtered= parole comuni tra glove e nostri tokens quindi automatimaticamente cancella le negation
glove_embeddings_filtered, glove_wo_pol= load_filter_glove_word(REPOSITORY+'glove.840B.300d.pkl',list(Data['Token']), CATEGORY, NEGATION_TYPE)

In [ ]:
len(glove_embeddings_filtered)

In [ ]:
Data=Data[Data['Token'].isin(glove_embeddings_filtered.keys())]

In [ ]:
Data.reset_index(drop=True, inplace=True)
Data

In [ ]:
Data['Embedding']=Data['Token'].apply(lambda x: glove_embeddings_filtered[x])

In [ ]:
Data.reset_index(drop=True, inplace=True)

In [ ]:
Data

In [ ]:
Dataset=SeedDataset(list(Data['Token']),glove_embeddings_filtered, pol=list(Data['Polarity']))

In [ ]:
trained_model = train(Dataset, CATEGORY, NEGATION_TYPE)

In [ ]:
all_vocabulary= Dataset.get_result()
non_seed_data={w:0 for w in glove_wo_pol.keys()}
non_seed_dataset = SeedDataset(list(non_seed_data.keys()),glove_wo_pol,split='test')

In [ ]:
results = predict(trained_model, non_seed_dataset, CATEGORY, NEGATION_TYPE)

In [ ]:
all_vocabulary.update(results)

In [ ]:
if NEGATION_MEAN:
  df_IMDB = pd.read_csv(REPOSITORY+'IMDB-Dataset_preprocessed.csv')
  accuracy_IMDB=experiment1(df_IMDB, complete_result, CATEGORY+'_'+NEGATION_TYPE+'_with_neg', 'IMDB')
else:
  df_IMDB = pd.read_csv(REPOSITORY+'IMDB-Dataset_preprocessed.csv')
  accuracy_IMDB=experiment1(df_IMDB, complete_result, CATEGORY+'_'+NEGATION_TYPE, 'IMDB')

In [ ]:
accuracy_IMDB

In [ ]:
if NEGATION_MEAN:
  df_tweets = pd.read_csv('/content/Sentiment_Lexicon/Data/Tweets_preprocessed.csv')
  accuracy_tweet=experiment1(df_tweets, complete_result, CATEGORY+'_'+NEGATION_TYPE+'_with_neg', 'Tweets')
else:
  df_tweets = pd.read_csv('/content/Sentiment_Lexicon/Data/Tweets_preprocessed.csv')
  accuracy_tweet=experiment1(df_tweets, complete_result, CATEGORY+'_'+NEGATION_TYPE, 'Tweets')
accuracy_tweet